For functional imaging experiments, trials were presented that consisted of the following stimuli, in non-randomized order: Black and white whole-field gratings were presented with motion in the forward direction at slow, medium, and fast speeds (3, 10, and 30 mm/s, respectively), for five seconds each with a pause of five seconds between stimuli, followed by reverse, leftward, and rightward moving gratings of the same duration and at medium speed. Grating remained static between stimuli. Black and white windmill patterns were rotated at 0.2 Hz with changing velocity that followed a sine function. Windmill patterns were presented across the whole field as well as for each half of the visual field. Flashes covered the whole visual field and switched between maximum luminance and darkness. For electrophysiological recordings, stimuli were similar as for functional imaging with the exception that the stimulus set had shorter pauses between stimuli and that fewer repetitions of the rotating windmill stimulus were presented. Blank trials consisting of static gratings were also interspersed with stimuli trials to obtain baseline responses. For one experiment (Figure 4—figure supplement 1g) the fish was also presented with a series of whole-field black or white flashes of various durations (50–5000 ms) against a baseline intermediate luminance.

In [34]:
## 上下


import numpy as np
import cv2

img_array=[]

for i in range(0,400):
    d = 400
    img = np.ones((d, d, 3), np.uint8) * 255

    center_x = 200
    center_y = i

    radius = 80

    cv2.circle(img, (center_x, center_y), radius, (0,0,0), -1)

    img_array.append(img)
   
out = cv2.VideoWriter('flash.avi',cv2.VideoWriter_fourcc(*'DIVX'), 400, (d,d))
for i in range(len(img_array)):     
    out.write(img_array[i])
    
out.release()
                      

In [22]:

## looming

import numpy as np
import cv2

img_array=[]

for i in range(0,400):
    d = 400
    img = np.ones((d, d, 3), np.uint8) * 255

    center_x = 200
    center_y = 200

    radius = 30
    radiu=int(2**(i/40))
    cv2.circle(img, (center_x, center_y), radiu, (0,0,0), -1)

    img_array.append(img)
    
    
def blank(inter,d,hz):#空白帧

    img_inter=[]
    for k in range(0,inter*hz):  
        img=np.ones((d, d, 3), np.uint8) * 255
        img_inter.append(img)
    return img_inter


cycle=img_array+blank(60,400,100)
final_img=[]
for q in range(0,5):
    final_img=final_img+cycle
    
final_img= blank(60,400,100) +final_img
   
out = cv2.VideoWriter('looming_i60-cycle-s4-i60.avi',cv2.VideoWriter_fourcc(*'DIVX'), 100, (d,d))
for i in range(len(final_img)):     
    out.write(final_img[i])


 
out.release()

In [83]:
import cv2
import numpy as np
from scipy import integrate

def windmill(frame_width, frame_height, fps, duration, num_blades, direction):
    num_frames = int(duration * fps)  # 总帧数
    frames = []  # 存储每一帧的列表

    def wind_ve(x):
        return np.sin(2 * 3.14159 * x / 5) * direction

    # 循环生成每一帧并记录在frames列表中
    for frame_count in range(num_frames):
        frame = np.ones((frame_width, frame_height, 3), np.uint8) * 0  # 创建黑色背景图像

        frame_t = frame_count / fps  # 计算当前帧的时间
        result, error = integrate.quad(wind_ve, 0, frame_t)  # 计算当前帧的位置
        angle = result / 3.14159 * 180  # 角速度乘以时间

        center_x = int(frame_width / 2)
        center_y = int(frame_height / 2)

        for j in range(0, 360, int(360 / num_blades * 2)):
            start_angle = -90 + angle
            cv2.ellipse(frame, (center_x, center_y), (center_x, center_y), 0, start_angle + j, start_angle + j + 20,
                        (255, 255, 255), -1)  # 绘制风车的叶片

        frames.append(frame)  # 将帧添加到列表中

    return frames

final = windmill(600, 600, 60, 10, 18, 1)

out = cv2.VideoWriter('testtttt.avi', cv2.VideoWriter_fourcc(*'DIVX'), 60, (600, 600))
for frame in final:
    out.write(frame)

out.release()

[array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8), array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,

In [104]:
import cv2
import numpy as np
from scipy import integrate

def windmill(frame_width, frame_height, fps, duration, num_blades, direction, mask='both'):
    num_frames = int(duration * fps)
    frames = []

    def wind_ve(x):
        return np.sin(2 * np.pi * x / 5) * direction

    for frame_count in range(num_frames):
        frame = np.zeros((frame_width, frame_height, 3), dtype=np.uint8)

        frame_t = frame_count / fps
        angle = integrate.quad(wind_ve, 0, frame_t)[0] * 180 / np.pi

        center = (frame_width // 2, frame_height // 2)
        for j in range(0, 360, int(360 / num_blades * 2)):
            start_angle = -90 + angle
            cv2.ellipse(frame, center, center, 0, start_angle + j, start_angle + j + 20, (255, 255, 255), -1)

        if mask == 'left':
            frame[:, frame_width // 2:] = 0
        elif mask == 'right':
            frame[:, :frame_width // 2] = 0

        frames.append(frame)

    return frames

def generate_black_white_frames(frame_width, frame_height, black_duration, white_duration, fps):
    num_black_frames = int(black_duration * fps)
    num_white_frames = int(white_duration * fps)

    # 创建黑色和白色帧
    black_frame = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
    white_frame = np.ones((frame_height, frame_width, 3), dtype=np.uint8) * 255

    frames = []

    # 生成白色帧
    for _ in range(num_white_frames):
        frames.append(white_frame.copy())

    # 生成黑色帧
    for _ in range(num_black_frames):
        frames.append(black_frame.copy())

    return frames


def grating(direction,v,t,hz,width,bar_width,real_width):  # width设为600比较好,V:cm/s,real-width:cm, direction=forward,backward,left,right
    
    grating_cycle=[]
    step=v/real_width*width/hz  #每帧应跨像素数     
    
    #最小循环单元
    n=0
    for i in np.arange(0,int(bar_width*2),step):
        n=n+1
        img = np.ones((width, width, 3), np.uint8) * 255
        for j in range(0,int(width/bar_width/2+2)):
            if direction=="forward":
                cv2.line(img, (0, int(j*2*bar_width-i+bar_width/2)), (width, int(j*2*bar_width-i+bar_width/2)), (0,0,0), bar_width)
            if direction=="reverse":
                cv2.line(img, (0, int((j-2)*2*bar_width+i+bar_width/2)), (width, int((j-2)*2*bar_width+i+bar_width/2)), (0,0,0), bar_width)
            if direction=="left":
                cv2.line(img, (int(j*2*bar_width-i+bar_width/2), 0), (int(j*2*bar_width-i+bar_width/2), width), (0,0,0), bar_width)   
            if direction=="right":
                cv2.line(img, (int((j-2)*2*bar_width+i+bar_width/2), 0), (int((j-2)*2*bar_width+i+bar_width/2), width), (0,0,0), bar_width)
        grating_cycle.append(img)
    
            
    #以最小循环单元为基础的循环数和余数
    cycle=int(t*hz/n)
    cycle_tail=t*hz % n

    img_arra=[]
    for q in range(0,cycle):  #
        img_arra=img_arra+grating_cycle
    img_sti=img_arra + grating_cycle[0:cycle_tail]
    
    return img_sti

def blank(inter,d,hz):  #空白帧 （时间，尺寸，帧率）
      # sec
    img_inter=[]
    for k in range(0,int(inter*hz)):  
        img=np.ones((d, d, 3), np.uint8) * 0
        img_inter.append(img)
    return img_inter


forward_03=grating("forward",v=0.3,t=5,hz=60,width=600,bar_width=60,real_width=5)
forward_1=grating("forward",v=1,t=5,hz=60,width=600,bar_width=60,real_width=5)
forward_3=grating("forward",v=3,t=5,hz=60,width=600,bar_width=60,real_width=5)
reverse=grating("reverse",v=1,t=5,hz=60,width=600,bar_width=60,real_width=5)
left=grating("left",v=1,t=5,hz=60,width=600,bar_width=60,real_width=5)
right=grating("right",v=1,t=5,hz=60,width=600,bar_width=60,real_width=5)
f_w=windmill(600, 600, 60, 5, 18, 1, mask='both')
r_w=windmill(600, 600, 60, 5, 18, -1, mask='both')
f_wR=windmill(600, 600, 60, 5, 18, 1, mask='right')
r_wR=windmill(600, 600, 60, 5, 18, -1, mask='right')
f_wl=windmill(600, 600, 60, 5, 18, 1, mask='left')
r_wl=windmill(600, 600, 60, 5, 18, -1, mask='left')
flash=generate_black_white_frames(600, 600, 1, 1, 60)
m_flash=flash+flash+flash+flash




Cycle = []

# 视频参数
output_width = 600
output_height = 600
fps = 60
duration_per_sequence = 5  # 每个序列的持续时间（秒）
gap_duration = 5  # 两个序列之间的间隔时间（秒）

# 将每个帧序列写入视频
sequences = [forward_03, forward_1, forward_3, reverse, left, right, f_w, r_w, f_wR, r_wl, m_flash]
for i, sequence in enumerate(sequences):
    num_frames = len(sequence)
    Cycle += sequence

    # 最后一个序列不生成间隔帧
    if i < len(sequences) - 2:
        # 间隔时间，显示上一个序列的最后一帧和下一个序列的第一帧
        gap_frames = int(gap_duration * fps)
        last_frame = sequence[-1]
        next_sequence = sequences[i + 1]
        next_frame = next_sequence[0]
        for _ in range(gap_frames // 2):
            Cycle.append(last_frame)
        for _ in range(gap_frames // 2):
            Cycle.append(next_frame)
    elif i == len(sequences) - 2:
        gap_frames = int(2.5 * fps)
        last_frame = sequence[-1]
        for _ in range(gap_frames):
            Cycle.append(last_frame)
        Cycle += blank(2.5,output_width,fps)
        
final = blank(10,output_width,fps) + Cycle + blank(10,output_width,fps) + Cycle + blank(10,output_width,fps)

# 创建 VideoWriter 对象
out = cv2.VideoWriter('output_final.avi', cv2.VideoWriter_fourcc(*'DIVX'), fps, (output_width, output_height))
# 保存视频
for frame in final:
    out.write(frame)
# 释放 VideoWriter 对象
out.release()









In [103]:
print(len(sequences))

11


## 光栅 旧版本，帧率随机

In [6]:
## 光栅 旧版本，帧率随机

import numpy as np
import cv2 

def grating(d,bar_width,vel,real_width,time):  #d像素长宽数目，bar,
    img_array=[]

    #定义最小循环单元, 一个bar移动后和初始图像完全重合的时间
    for i in range(0,int(bar_width*2)):
        img = np.ones((d, d, 3), np.uint8) * 255
        #print(img)

        for j in range(0,int(d/bar_width)):
            cv2.line(img, (j*2*bar_width-i+int(bar_width/2), 0), (j*2*bar_width-i+int(bar_width/2), d), (0,0,0), bar_width)  #左上顶点和右下顶点，颜色，线条宽度
        
        img_array.append(img)
        
    hz=  int(vel/real_width*d)


    cycle=int(time*hz/bar_width/2)
    cycle_tail=time*hz % (bar_width*2)

    img_arra=[]
    for q in range(0,cycle):  # 100倍相当于100个循环，一个循环i帧，也就是一个循环图像不变。应该够了
        img_arra=img_arra+img_array
    img_sti=img_arra + img_array[0:cycle_tail]
    
    return img_sti,hz

def blank(inter,d,hz):  #空白帧
      # sec
    img_inter=[]
    for k in range(0,inter*hz):  
        img=np.ones((d, d, 3), np.uint8) * 255
        img_inter.append(img)
    return img_inter

a,b=grating(600,60,3,4,5)
c=blank(20,600,b)

cycle=a+c
final_img=[]
for q in range(0,10):
    final_img=final_img+cycle

final_img=blank(60,600,b)+final_img

out = cv2.VideoWriter('whole-field flashes_60_cycle10-v3-5-20.avi',cv2.VideoWriter_fourcc(*'DIVX'), b, (600,600))    #
for i in range(len(final_img)):     
    out.write(final_img[i])
    
 
out.release()

## 光栅混合——V1

In [4]:
#grating 速度，方向，bar大小，帧率均可调节，支持多种模式混合拼接，后续增加looming, windmill

import numpy as np
import cv2 

def grating(direction,v,t,hz,width,bar_width,real_width):  # width设为600比较好,V:cm/s,real-width:cm, direction=forward,backward,left,right
    
    grating_cycle=[]
    step=v/real_width*width/hz  #每帧应跨像素数     
    
    #最小循环单元
    n=0
    for i in np.arange(0,int(bar_width*2),step):
        n=n+1
        img = np.ones((width, width, 3), np.uint8) * 255
        for j in range(0,int(width/bar_width/2+2)):
            if direction=="forward":
                cv2.line(img, (0, int(j*2*bar_width-i+bar_width/2)), (width, int(j*2*bar_width-i+bar_width/2)), (0,0,0), bar_width)
            if direction=="reverse":
                cv2.line(img, (0, int((j-2)*2*bar_width+i+bar_width/2)), (width, int((j-2)*2*bar_width+i+bar_width/2)), (0,0,0), bar_width)
            if direction=="left":
                cv2.line(img, (int(j*2*bar_width-i+bar_width/2), 0), (int(j*2*bar_width-i+bar_width/2), width), (0,0,0), bar_width)   
            if direction=="right":
                cv2.line(img, (int((j-2)*2*bar_width+i+bar_width/2), 0), (int((j-2)*2*bar_width+i+bar_width/2), width), (0,0,0), bar_width)
        grating_cycle.append(img)
    
            
    #以最小循环单元为基础的循环数和余数
    cycle=int(t*hz/n)
    cycle_tail=t*hz % n

    img_arra=[]
    for q in range(0,cycle):  #
        img_arra=img_arra+grating_cycle
    img_sti=img_arra + grating_cycle[0:cycle_tail]
    
    return img_sti

    
def blank(inter,d,hz):  #空白帧 （时间，尺寸，帧率）
      # sec
    img_inter=[]
    for k in range(0,inter*hz):  
        img=np.ones((d, d, 3), np.uint8) * 255
        img_inter.append(img)
    return img_inter


def static(static,t,hz):
    static_img=[]
    for i in range(0,t*hz):  
        static_img.append(static)
    return static_img
 


In [ ]:

#模式输出序列，间隔为白色空白
Cycle=grating("forward",v=0.3,t=5,hz=60,width=600,bar_width=60,real_width=4) + blank(5,600,60) \
    + grating("forward",v=1,t=5,hz=60,width=600,bar_width=60,real_width=4) \
    + blank(10,600,60) \
    + grating("forward",v=3,t=5,hz=60,width=600,bar_width=60,real_width=4) \
    + blank(10,600,60) \
    + grating("reverse",v=1,t=5,hz=60,width=600,bar_width=60,real_width=4) \
    + blank(10,600,60) \
    + grating("left",v=1,t=5,hz=60,width=600,bar_width=60,real_width=4) \
    + blank(10,600,60) \
    + grating("right",v=1,t=5,hz=60,width=600,bar_width=60,real_width=4) \
    + blank(10,600,60)


#模式输出序列的 重复trail
multi_trial=[]
for q in range(0,3):   
    multi_trial=multi_trial+Cycle
multi_trial=blank(30,600,60) + multi_trial

out = cv2.VideoWriter('grating_B-30s_Cycle3_inter-10s_F0.3_F1_F3_R1_Le1_Ri1.avi',cv2.VideoWriter_fourcc(*'DIVX'), 60, (600,600))    #请匹配以上帧率 和 图像大小
    
for i in range(len(multi_trial)):     
    out.write(multi_trial[i])
out.release() 
    


In [6]:
# 以grating 静态作为间隔 模式输出
forward_03=grating("forward",v=0.3,t=5,hz=60,width=600,bar_width=60,real_width=4)
forward_1=grating("forward",v=1,t=5,hz=60,width=600,bar_width=60,real_width=4)
forward_3=grating("forward",v=3,t=5,hz=60,width=600,bar_width=60,real_width=4)
reverse=grating("reverse",v=1,t=5,hz=60,width=600,bar_width=60,real_width=4)
left=grating("left",v=1,t=5,hz=60,width=600,bar_width=60,real_width=4)
right=grating("right",v=1,t=5,hz=60,width=600,bar_width=60,real_width=4)

Cycle= []
for i in ["forward_03","forward_1","forward_3","reverse","left","right"]:
    Cycle=Cycle + eval(i)+static(eval(i)[-1],10,60)

#模式输出序列的 重复trail
multi_trial=[]
for q in range(0,3):   
    multi_trial=multi_trial+Cycle
multi_trial=static(forward_03[0],30,60) + multi_trial

out = cv2.VideoWriter('grating_Static-30s_Cycle5_inter-10s_F0.3_F1_F3_R1_Le1_Ri1.avi',cv2.VideoWriter_fourcc(*'DIVX'), 60, (600,600))    #请匹配以上帧率 和 图像大小
    
for i in range(len(multi_trial)):     
    out.write(multi_trial[i])
out.release() 